In [ ]:
import glob
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# PART 1: Retrieval
LOAD_WHOLE_DATASET = True

# See data/REAMD.md for longer names/details.
columns = ['x', 'y', 'z', 'label']
label_i = [1, 2, 3, 4, 5, 6, 7]
person_i = [i for i in range(1, 16)]
labels = ['Computer', 'Moving', 'Standing', 'Walking', 'Stairs',
          'Walking + talking', 'Standing + talking']
labels_short = ['Computer', 'Move', 'Stand', 'Walk', 'Stairs',
          'Walk+talk', 'Stand+talk']
# Map numeric label to user friendly names
labels_map = {
    1: 'Computer',
    2: 'Moving',
    3: 'Standing',
    4: 'Walking',
    5: 'Stairs',
    6: 'Walking + talking',
    7: 'Standing + talking'
}

accs = ['x', 'y', 'z']
acc_pairs = [('x', 'y'), ('x', 'z'), ('y', 'z')]


label_color = {
    'Computer': 'red',
    'Moving': 'green',
    'Standing': 'black',
    'Walking': 'yellow',
    'Stairs': 'cyan',
    'Walking + talking': 'orange',
    'Standing + talking': 'purple' ,
}


ad = DataFrame()
# find all csv files
if LOAD_WHOLE_DATASET:
    files = glob.glob('data/*.csv')
else:
    files = ['data/2.csv']

# concatenate all 15 data files
# usecols is used to skip reading the 'id' column, which has errors
for file in files:
    person_data = pd.read_csv(file, sep=',', header=None, names=columns,
                 usecols=[1,2,3,4])
    # Add a column person, which specifies which dataset a row is from
    person_id = file.split('data/')[1].split('.csv')[0]
    person_data['person'] = int(person_id)

    ad = pd.concat([ad, person_data])


In [ ]:
# Drop rows with not a valid label
ad = ad.loc[(ad['label'] >= 1) & (ad['label'] <= 7), :]

In [ ]:
# Add new columns, readable label column and color
ad['label_readable'] = ad['label'].map(labels_map)

# add a color to each class (label)
ad['color'] = ad['label_readable'].map(label_color)


In [ ]:
# check data
print('\nShape')
print(ad.shape)
# Check head and tail
print('\nHead')
print(ad.head())
print('\nTail')
print(ad.tail())
# Check if any value is NaN, if this was the case those rows would need to be cleaned.
print('\nColumns with NaNs')
print(ad.isna().any())

In [ ]:
# Investigate data, show data type, max/min, label count.
for column in columns:
    print()
    print(f"Column: {column}, dtype: {ad[column].dtype}")

    if column == 'label':
        print(f"Label count:\n{ad[column].value_counts()}")
    elif column in accs:
        print(column, 'min, max:', ad[column].min(), ad[column].max())
        print(column, 'mean, std:', ad[column].mean(), ad[column].std())

        print('Sorted head:')
        print(ad[column].sort_values().head(10))
        print('Sorted tail:')
        print(ad[column].sort_values().tail(10))

# From the output above we can see the data is unbalanced, 'moving' has 928 data points,
# while 'standing + walking' has 83748. When using knn, the feature with most
# data points will be favoured.

# We also see that y, z have data points which seem faulty. There are a few of
# abnormally small and big values (e.g. 2 and 4095).

# We can investigate the abnormal y and z values further:
outs_y = ad.loc[(ad['y'] > 3800) | (ad['y'] < 200), :]
outs_z = ad.loc[(ad['z'] > 3800) | (ad['z'] < 200), :]
print('\ny outliers:')
print(outs_y.sort_values(by='y'))
print('z outliers:')
print(outs_z.sort_values(by='z'))
# We see they often correlate, a small y follows a big z, and vice versa,
# but not always. All are for 'Computer' class

In [ ]:
# I will investigate further, and look at the distribution
# of the outliers. 
outs_yh = ad.loc[(ad['y'] > 2900), 'y']
outs_yl = ad.loc[(ad['y'] < 1750), 'y']
outs_zh = ad.loc[(ad['z'] > 2500), 'z']
outs_zl = ad.loc[(ad['z'] < 1500), 'z']


# In the figure below we can see the frequency distribution 
# for y in the range above 2900. This shows the outliers mentioned above, 
# and clearly shows that they differ from the data points around it. 
# Similar graphs are made for low y-values, high and low z-values. 
outs_yh.plot.hist(bins=100)
plt.title('y high outliers frequency')
plt.xlabel('y high')
#plt.savefig('pic/y_high_outlier')
plt.show()


outs_yl.plot.hist(bins=100)
plt.title('y low outliers frequency')
plt.xlabel('y low')
plt.show()


outs_zh.plot.hist(bins=100)
plt.title('z high outliers frequency')
plt.xlabel('z high')
plt.show()


outs_zl.plot.hist(bins=100)
plt.title('z low outliers frequency')
plt.xlabel('z low')
plt.show()


    

In [ ]:
# PART 2: Data Exploration

In [ ]:
# Generate a scatter matrix to get a quick overview
scatter_matrix(ad, alpha=.2, figsize=(16,16), diagonal='kde')
#plt.savefig('pic/scattermatrix.png')


In [ ]:
# Explore acceleration data point distribution
def explore_acc(feature='x'):
    print('Exploring feature: ', feature)
    feat = ad[feature]
    print(f"{feature} min: {ad[feature].min()}, max: {ad[feature].max()}")
    # show frequency distribution
    print(feat.value_counts(bins=20, normalize=True, sort=False))
    # plot distribution
    print(feat.plot.hist(bins=100))
    plt.xlabel(f"{feature} acceleration")
    plt.ylabel('Frequency')

    plt.title(f"{feature} acceleration frequency distribution")
    plt.tight_layout()

    plt.savefig(f"pic/{feature}-histogram")
    plt.show()

for acc in accs:
    explore_acc(acc)

In [ ]:
# Explore class types and distribution 
fig, ax = plt.subplots()
ad['label_readable'].value_counts().plot.bar()
ax.set_xticklabels(labels_short)
plt.title('Class frequency')
plt.ylabel('Frequency')
plt.tight_layout()
plt.xticks(rotation=45)
plt.savefig('pic/class-bar-chart')

In [ ]:
# Explore pair of attributes (columns)
# plot class against each acceleration
for acc in accs:
    fig, ax = plt.subplots(figsize=(8,6))
    ad.boxplot(column=acc, by='label_readable', ax=ax, grid=False)
    ax.set_xticklabels(labels_short)
    plt.xlabel('Activity')
    plt.ylabel('Acceleration value')
    plt.title(f"Distribution of {acc} acceleration for each class of movement")
    plt.suptitle('')
    #plt.savefig(f"pic/{acc}-class-boxplot")
    fig.show()
    #break


In [ ]:
# Shows distribution of accelerations for each activity.
for label in labels:
    label_acc = ad.loc[ad['label_readable'] == label, accs]
    fig, ax = plt.subplots()
    label_acc.boxplot(grid=False)
    plt.xlabel('Acceleration axis')
    plt.ylabel('Acceleration value')
    plt.title(f"Distribution of {label} acceleration for each axis of acceleration")
    plt.suptitle('')
    plt.show()

In [ ]:
# Hypothesis: I expect to see some relationship between accelerations, eg.
# walking in stairs would have both x acc and y acc higher, but walking flat would
# have little change in y, but maybe more in z.
# Result: I can see some groupings/clusters forming, but I cannot see which
# classes are forming these clusters. Need to color code
for acc in accs:
    for acc2 in accs:
        if acc == acc2: continue
        ad.plot.scatter(x=acc, y=acc2)
        plt.title(f"Acceleration: {acc} vs {acc2}")
        plt.xlabel(f"{acc}")
        plt.ylabel(f"{acc2}")
        plt.show()

In [ ]:
# TASK: Explore the relationship between all pairs of attributes
# Compare class label, vs accelerations
# Hypothesis: my hypothesis is that activities where the participants move will have the most variance.  
# Result: This type of graph is not suited, it gives unexpected and most likely wrong result

acc_pairs = [('x', 'y'), ('x', 'z'), ('y', 'z')]
for acc1, acc2 in acc_pairs:
    fig, ax = plt.subplots()
    ax.scatter(x=ad[acc1], y=ad[acc2], c=ad['color'], alpha=0.4)
    plt.title(f"Acceleration: {acc1} vs {acc2} by activity")
    plt.xlabel(f"{acc1}")
    plt.ylabel(f"{acc2}")
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    #retrieve values from color dictionary and attribute it to corresponding labels
    leg_el = [mpatches.Patch(facecolor = value, edgecolor = "black", label = key, alpha = 0.4) for key, value in label_color.items()]
    
    # Put a legend to the right of the current axis
    ax.legend(handles = leg_el, loc='center left', bbox_to_anchor=(1, 0.5))
    #plt.savefig(f"pic/{acc1}-{acc2}-class-colour-scatter-(all: {LOAD_SINGLE_DATASET})")
    plt.show()

In [ ]:
# Plot line chart for each person, for each activity, with all accelerations in one graph
for p in range(1, 16):
    for activity in labels:
        fig, ax = plt.subplots()
        plt.title(f"Person: {p}, Activity: {activity}")
        d  = ad.loc[(ad['person'] == p) & (ad['label_readable'] == activity), accs].reset_index()
        for acc in accs:
            ds = d[acc]
            ds.plot(label=acc)
        plt.legend()
        plt.ylim(0, 4100)
        plt.xlabel('Sequential measurement nr.')
        plt.ylabel('Acceleration')
        plt.show()


In [ ]:
# Plot line chart for each activity (combine all persons), with all accelerations in one graph
for activity in labels:
    fig, ax = plt.subplots()
    plt.title(f"Activity: {activity}")
    d  = ad.loc[ad['label_readable'] == activity, accs].reset_index()
    for acc in accs:
        ds = d[acc]
        ds.plot(label=acc)
    plt.legend()
    plt.ylim(0, 4100)
    plt.xlabel('Sequential measurement nr.')
    plt.ylabel('Acceleration')
    plt.show()


In [ ]:
# Plot line chart for pair of accelerations, for each activity

for activity in labels:
    d  = ad.loc[ad['label_readable'] == activity, accs].reset_index()
    for acc1, acc2 in acc_pairs:
        ds = d[[acc1, acc2]].plot()
        
        plt.title(f"{activity}, acceleration {acc1} vs. {acc2}")
        plt.legend()
        plt.ylim(0, 4100)
        plt.xlabel('Sequential measurement nr.')
        plt.ylabel('Acceleration')
        plt.tight_layout()
        plt.savefig(f"pic/{activity}-{acc1}-{acc2}")
        plt.show()
        


In [ ]:
# Plot line chart for pairs of 1 acceleration and 1 activity
for activity in labels:
    d  = ad.loc[ad['label_readable'] == activity, accs].reset_index()
    for acc in accs:
        d[acc].plot(label=acc)
        plt.title(f"Activity: {activity}, acceleration {acc}")
        plt.legend()
        plt.ylim(0, 4100)
        plt.xlabel('Sequential measurement nr.')
        plt.ylabel('Acceleration')
        plt.tight_layout()
        plt.savefig(f'pic/{activity}-{acc}')
        plt.show()

In [ ]:
# 3D plot
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.scatter(ad['x'], ad['y'], ad['z'], c=ad['color'], alpha=.2)
plt.title(f"Acceleration: x, y, z by class")
ax.set_xlabel(f"x acceleration ")
ax.set_ylabel(f"y acceleration ")
ax.set_zlabel('z acceleration ')
leg_el = [mpatches.Patch(facecolor = value, edgecolor = "black", label = key, alpha = 0.4) for key, value in label_color.items()]
# Put a legend to the right of the current axis
ax.legend(handles = leg_el, loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig(f"3d-scatter")
plt.show()

In [ ]:
# PART 3: MODELING
# The goal of the modeling is to see if it is possible to classify an unknown
# activity (one or a set of data points). This is classification, so possible
# models are knn, decision tree.

In [17]:
# Extract features for data modeling
# takes dataframe for 1 person, windows it, and returns a new dataframe with new features
# wsize defaults to 1 second (=52 Hz)
# returns mean, std, max-min,
def windowing(df, wsize=52):
    person = df['person'][0] # get person id
    df_list = []  # storing dicts in a list then creating dataframe in the end is much faster
    
    # iterate over all activities
    for label in label_i:
        # extract label activity
        df_t = df.loc[df['label'] == label, :].reset_index()
        label_readable, color = df_t['label_readable'][0], df_t['color'][0]
        
        slide = 0  # slide will slide over the time series, increasing wsize//2 each iteration
        window = wsize
        size = len(df_t.index)
        while slide < size:
            # extract window frame
            rows = df_t.iloc[slide:window][accs]
            if len(rows) < 2: break
            df_list.append({
                'person': person,
                'label': label,
                'label_readable': label_readable,
                'color': color,
                'x mean': rows['x'].mean(),
                'y mean': rows['y'].mean(),
                'z mean': rows['z'].mean(),
                'x std': rows['x'].std(),
                'y std': rows['y'].std(),
                'z std': rows['z'].std(),
                'x diff': rows['x'].max() - rows['x'].min(),
                'y diff': rows['y'].max() - rows['y'].min(),
                'z diff': rows['z'].max() - rows['z'].min(),
                })
            # move window
            window = slide + wsize
            slide += wsize//2


    return pd.DataFrame(df_list)
        
ad_l = []
for person in person_i:
    print(f"Computing for person {person}")
    ad_l.append(windowing(ad.loc[ad['person'] == person, :]))
ad_n = pd.concat(ad_l, ignore_index=True)
ad_n

Computing for person 1
Computing for person 2
Computing for person 3
Computing for person 4
Computing for person 5
Computing for person 6
Computing for person 7
Computing for person 8
Computing for person 9
Computing for person 10
Computing for person 11
Computing for person 12
Computing for person 13
Computing for person 14
Computing for person 15


,person,label,label_readable,color,x mean,y mean,z mean,x std,y std,z std,x diff,y diff,z diff
0,1,1,Computer,red,1598.923077,2001.673077,2004.000000,39.374943,115.800491,71.632504,192,771,322
1,1,1,Computer,red,1609.615385,1964.192308,2046.307692,28.684598,61.166016,46.296237,133,349,206
2,1,1,Computer,red,1585.461538,2022.846154,2086.615385,50.308831,42.165097,52.471003,199,152,179
3,1,1,Computer,red,1668.730769,2175.153846,2045.769231,43.481083,86.539097,95.318963,171,345,412
4,1,1,Computer,red,1738.076923,2240.769231,1914.153846,115.771473,93.420044,200.639915,391,498,1068
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74012,15,7,Standing + talking,purple,2052.500000,2531.884615,1988.461538,51.246268,33.838826,34.081351,185,137,121
74013,15,7,Standing + talking,purple,2059.038462,2539.615385,2011.500000,7.507227,22.369760,9.749872,32,131,40
74014,15,7,Standing + talking,purple,2043.115385,2526.384615,2004.384615,19.290053,14.716187,11.289205,62,53,46
74015,15,7,Standing + talking,purple,2043.884615,2529.846154,1996.461538,15.074022,28.794016,9.157427,68,120,37


In [ ]:
# Inspect features extracted
# Compare means in scatter plot
acc_pairs = [('x mean', 'y mean'), ('x mean', 'z mean'), ('y mean', 'z mean')]
for acc1, acc2 in acc_pairs:
    fig, ax = plt.subplots()
    ax.scatter(x=ad_n[acc1], y=ad_n[acc2], c=ad_n['color'], alpha=0.4)
    plt.title(f"Acceleration: {acc1} vs {acc2} by activity (windowed)")
    plt.xlabel(f"{acc1}")
    plt.ylabel(f"{acc2}")
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    #retrieve values from color dictionary and attribute it to corresponding labels
    leg_el = [mpatches.Patch(facecolor = value, edgecolor = "black", label = key, alpha = 0.4) for key, value in label_color.items()]
    
    # Put a legend to the right of the current axis
    ax.legend(handles = leg_el, loc='center left', bbox_to_anchor=(1, 0.5))
    #plt.savefig(f"{acc1}-{acc2}-class-colour-scatter-(all: {LOAD_SINGLE_DATASET})")
    plt.show()

In [ ]:
# Inspect features extracted
# Boxplot of means
means = ['x mean', 'y mean', 'z mean']
for mean in means:
    fig, ax = plt.subplots(figsize=(8,6))
    ad_n.boxplot(column=mean, by='label_readable', ax=ax, grid=False)
    ax.set_xticklabels(labels_short)
    plt.xlabel('Activity')
    plt.ylabel('Acceleration mean value (windowed)')
    plt.title(f"Distribution of {mean} acceleration for each class of movement")
    plt.suptitle('')
    #plt.savefig(f"{mean}-class-boxplot")
    fig.show()
    #break

In [ ]:
# Inspect features extracted
# 3D plot
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.scatter(ad_n['x mean'], ad_n['y mean'], ad_n['z mean'], c=ad_n['color'], alpha=.2)
plt.title(f"Acceleration: x, y, z  mean by class (windowed)")
ax.set_xlabel(f"x acceleration mean")
ax.set_ylabel(f"y acceleration mean")
ax.set_zlabel('z acceleration mean')
leg_el = [mpatches.Patch(facecolor = value, edgecolor = "black", label = key, alpha = 0.4) for key, value in label_color.items()]
# Put a legend to the right of the current axis
ax.legend(handles = leg_el, loc='center left', bbox_to_anchor=(1, 0.5))

#plt.savefig(f"3d-scatter-mean-windowed")
plt.show()


In [97]:
# Define X - feature rows, y - label rows
X = ad_n.iloc[:, 4:]
y = ad_n.loc[:, 'label']

In [ ]:
# Feature selection with hill climbing
from sklearn.utils import shuffle

# pass in feature columns, target column, and a classifier
def feature_hill_climbing(data, target, classifier):

    data_columns = data.columns
    num_features = len(data_columns)

    #col_num = data.shape[1] # number of data columns
    new_index = []
    max_score = 0.0
    feature_index_random = shuffle(data_columns, random_state=39)

    clf = classifier

    for feature_index in range(num_features):
        new_index.append(feature_index_random[feature_index])
        new_data = data.loc[:, new_index]
        X_train, X_test, y_train, y_test = train_test_split(
            new_data, target, test_size=0.4, random_state=2)

        fit = clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        if score < max_score:
            new_index.remove(feature_index_random[feature_index])
            print('removed', feature_index_random[feature_index])
        else:
            max_score = score
            print(f"Score with {len(new_index)} selected features: {score}")
    print('Selected features:')
    print(sorted(new_index))


In [87]:
knnc = KNeighborsClassifier(6, weights='distance', p=1)
dtc = DecisionTreeClassifier(max_depth=5, min_samples_split=100)

feature_hill_climbing(X, y, dtc)

# knn: use all 9
# decision tree: use
# use all 9

Score with 1 selected features: 0.4802242712871956
Score with 2 selected features: 0.5106900395176817
Score with 3 selected features: 0.6624784679298814
Score with 4 selected features: 0.6628500016887898
Score with 5 selected features: 0.6628837774850542
Score with 6 selected features: 0.6661600297227007
Score with 7 selected features: 0.6661600297227007
Score with 8 selected features: 0.6667342182591954
Score with 9 selected features: 0.6803458641537474
Selected features:
['x diff', 'x mean', 'x std', 'y diff', 'y mean', 'y std', 'z diff', 'z mean', 'z std']


In [109]:
# K-Folds cross validation
from sklearn.model_selection import KFold
def k_folds(data, target, classifier):
    sum = 0
    kf = KFold(n_splits=10, random_state=0)
    for k, (train_index, test_index) in enumerate(kf.split(data)):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]

        fit = classifier.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        sum += score
        print(f"[Fold {k}] score: {score}")
    print(f"Average of {k} folds: {sum/(k+1)}")

In [110]:
# Test folds
knnc = KNeighborsClassifier(n_neighbors=4)
dtc = DecisionTreeClassifier(max_depth=5, min_samples_split=100)
k_folds(X, y, knnc)

/home/samuelsen/.local/share/virtualenvs/Assignment2-QHtghdoQ/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


[Fold 0] score: 0.6703593623345042
[Fold 1] score: 0.5660632261550932
[Fold 2] score: 0.6272629019184004
[Fold 3] score: 0.7807349365036477
[Fold 4] score: 0.7415563361253715
[Fold 5] score: 0.735341799513645
[Fold 6] score: 0.6752229127262902
[Fold 7] score: 0.5973517092284827
[Fold 8] score: 0.678692068639373
[Fold 9] score: 0.6776111336305904
Average of 9 folds: 0.6750196386775398


In [90]:
# KNN
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
neighbours = 3
clf = KNeighborsClassifier(n_neighbors=neighbours)
fit = clf.fit(X_train, y_train)
y_pred = fit.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.91      0.93      0.92      5778
           2       0.50      0.40      0.44       502
           3       0.61      0.59      0.60      2094
           4       0.81      0.89      0.85      3410
           5       0.53      0.33      0.41       518
           6       0.62      0.36      0.45       464
           7       0.86      0.87      0.87      5739

    accuracy                           0.82     18505
   macro avg       0.69      0.62      0.65     18505
weighted avg       0.81      0.82      0.82     18505



In [95]:
# Decision tree

clf = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=100
)
fit = clf.fit(X_train, y_train)
y_pred = fit.predict(X_test)
confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.73      0.81      0.77      5778
           2       0.60      0.05      0.09       502
           3       0.43      0.02      0.03      2094
           4       0.76      0.87      0.81      3410
           5       0.58      0.07      0.13       518
           6       0.00      0.00      0.00       464
           7       0.58      0.81      0.67      5739

    accuracy                           0.67     18505
   macro avg       0.52      0.37      0.36     18505
weighted avg       0.63      0.67      0.61     18505



/home/samuelsen/.local/share/virtualenvs/Assignment2-QHtghdoQ/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
from sklearn import tree


with open('activity.dot', 'w') as f:
    f = tree.export_graphviz(clf, out_file=f, feature_names=X.columns, class_names=labels,
                             filled=True, rounded=True, special_characters=True)



